<a href="https://colab.research.google.com/github/Praise-Atadja/CognitiveQuest/blob/main/Transfer_Learning_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **PROJECT NAME: CognitiveQuest**
---
(***This project is to predict the possibility of autism***)

# **OBJECTIVE**

This project aims to use transfer learning techniques to predict Autism Spectrum Disorder (ASD) from image data. This involves fine-tuning pre-trained neural network models on a dataset related to ASD. The steps include preprocessing the images, selecting a suitable pre-trained model, adjusting its parameters, and evaluating its performance using accuracy, precision, recall, and F1-score metrics.

## **The Dataset**


**Short Description of the Data:**

This dataset consists of images categorized into two classes: "autistic" and "non_autistic", aiming to facilitate research and analysis in understanding autism spectrum traits. The dataset is divided into three main folders: "train", "valid", and "test", each containing two subfolders representing the two classes.

**Folder Structure:**

train: Contains 1263 images in each of the "autistic" and "non_autistic" subfolders, totaling 2526 images.

valid: Contains 100 images in each of the "autistic" and "non_autistic" subfolders, totaling 200 images.

test: Contains 100 images in each of the "autistic" and "non_autistic" subfolders, totaling 200 images.

In [2]:
#Import Necessary Libraries
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import tensorflow as tf
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.applications import VGG16, ResNet50, InceptionV3
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Loading data

In [3]:
# dataset directories
train_dir = "/content/drive/MyDrive/dataset/train"
valid_dir = "/content/drive/MyDrive/dataset/valid"
test_dir = "/content/drive/MyDrive/dataset/test"

def load_data(train_dir, valid_dir, test_dir, img_height=224, img_width=224, batch_size=32):
    train_datagen = ImageDataGenerator(
        rescale=1./255,
        rotation_range=20,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest'
    )

    valid_datagen = ImageDataGenerator(rescale=1./255)
    test_datagen = ImageDataGenerator(rescale=1./255)

    train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=(img_height, img_width),
        batch_size=batch_size,
        class_mode='categorical'
    )

    valid_generator = valid_datagen.flow_from_directory(
        valid_dir,
        target_size=(img_height, img_width),
        batch_size=batch_size,
        class_mode='categorical'
    )

    test_generator = test_datagen.flow_from_directory(
        test_dir,
        target_size=(img_height, img_width),
        batch_size=batch_size,
        class_mode='categorical'
    )

    return train_generator, valid_generator, test_generator

train_generator, valid_generator, test_generator = load_data(train_dir, valid_dir, test_dir)

Found 2575 images belonging to 2 classes.
Found 200 images belonging to 2 classes.
Found 200 images belonging to 2 classes.


In [4]:
#Base Model

def build_model(base_model, num_classes=2):
    x = Flatten()(base_model.output)
    x = Dense(128, activation='relu')(x)
    predictions = Dense(num_classes, activation='softmax')(x)
    model = Model(inputs=base_model.input, outputs=predictions)
    return model

def train_model(model, train_generator, valid_generator, epochs=10):
    model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
    history = model.fit(train_generator, validation_data=valid_generator, epochs=epochs)
    return history

def evaluate_model(model, test_generator):
    y_true = test_generator.classes
    y_pred = model.predict(test_generator)
    y_pred_classes = np.argmax(y_pred, axis=1)

    accuracy = accuracy_score(y_true, y_pred_classes)
    loss = model.evaluate(test_generator)[0]
    precision = precision_score(y_true, y_pred_classes)
    recall = recall_score(y_true, y_pred_classes)
    f1 = f1_score(y_true, y_pred_classes)

    return accuracy, loss, precision, recall, f1

##Loading Pre-trained Model

In [5]:
# Load pre-trained models
vgg_base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
resnet_base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
inception_base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Fine-tuning model

In [6]:
# Unfreeze some top layers of the pre-trained models for fine-tuning
for layer in vgg_base_model.layers[-4:]:
    layer.trainable = True

for layer in resnet_base_model.layers[-10:]:
    layer.trainable = True

for layer in inception_base_model.layers[-15:]:
    layer.trainable = True


In [7]:
# Compile the models
vgg_model = build_model(vgg_base_model)
resnet_model = build_model(resnet_base_model)
inception_model = build_model(inception_base_model)

vgg_model.compile(optimizer=Adam(lr=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])
resnet_model.compile(optimizer=Adam(lr=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])
inception_model.compile(optimizer=Adam(lr=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# Train the models
vgg_finetune_history = train_model(vgg_model, train_generator, valid_generator, epochs=5)
resnet_finetune_history = train_model(resnet_model, train_generator, valid_generator, epochs=5)
inception_finetune_history = train_model(inception_model, train_generator, valid_generator, epochs=5)


Epoch 1/5
81/81 [==============================] - 5857s 72s/step - loss: 2.7502 - accuracy: 0.5076 - val_loss: 0.6930 - val_accuracy: 0.5000
Epoch 2/5
72/81 [=========================>....] - ETA: 10:25 - loss: 0.6965 - accuracy: 0.5002

# Evaluating the model

In [ ]:
# Evaluate the fine-tuned models
vgg_finetune_eval_metrics = evaluate_model(vgg_model, test_generator)
resnet_finetune_eval_metrics = evaluate_model(resnet_model, test_generator)
inception_finetune_eval_metrics = evaluate_model(inception_model, test_generator)

In [ ]:
# Print evaluation metrics for tuned models
print("VGG16 Fine-tuned Model Evaluation:", vgg_finetune_eval_metrics)
print("ResNet50 Fine-tuned Model Evaluation:", resnet_finetune_eval_metrics)
print("InceptionV3 Fine-tuned Model Evaluation:", inception_finetune_eval_metrics)
